<a href="https://colab.research.google.com/github/julianolm/Local-Binary-Convolutional-Neural-Network/blob/main/LBCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of LBCNN and application on CIFAR-10

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

In [2]:
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f'Training on device {device}.')

Training on device cuda.


## Preparing the Dataset

In [4]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
cifar10_val = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


## Model

In [5]:
def new_kernel(in_channels=384, inter_channels=512, sparsity=0.1):
    """
        Returns the anchor weights for a layer with "in_channels" in channels and one output channel.
    """
    from random import randint
    from math import ceil

    nk = np.zeros((inter_channels, in_channels*3*3))
    for out in range(inter_channels):
        for i in range(ceil(in_channels*3*3*sparsity)):
            a = randint(0,in_channels*3*3 - 1)
            nk[out, a] = 1 if randint(1,2)==1 else -1

    nk = nk.reshape(inter_channels, in_channels, 3, 3)
    return nk

In [6]:
class LBCBlock(nn.Module):
    def __init__(self, n_channels=384, n_kernels=512, sparsity=0.1):
        super().__init__()
        self.n_channels = n_channels
        self.n_kernels = n_kernels
        self.Batch_Norm = nn.BatchNorm2d(n_channels)

        self.conv_filter = nn.Conv2d(n_channels, n_kernels, kernel_size=3, padding=1, bias=False)
        kernels = torch.tensor(new_kernel(n_channels, n_kernels, sparsity)).type('torch.FloatTensor')
        self.conv_filter.weight = nn.Parameter(kernels, requires_grad=False)

        self.weighted_sum = nn.Conv2d(n_kernels, n_channels, kernel_size=1, bias=False)

    def forward(self, x):
        out = self.Batch_Norm(x)
        with torch.no_grad():
            out = torch.relu(self.conv_filter(out))
        out = self.weighted_sum(out)
        out += x

        return out

In [7]:
img_width = 32
img_height = 32

class NetLBC(nn.Module):
    def __init__(self, lbc_filters=512, n_channels=384, n_blocks=10, sparsity=0.1, hidden_units=384):
        super().__init__()
        self.n_channels = n_channels
        self.n_blocks = n_blocks
        self.hidden_units = hidden_units

        self.conv1 = nn.Conv2d(3, n_channels, kernel_size=3, padding=1)
        
        from collections import OrderedDict
        self.lbc_blocks = nn.Sequential(
            OrderedDict([
                         (f'lbc_block_{i}', LBCBlock(n_channels, lbc_filters, sparsity)) for i in range(1,n_blocks+1) 
                         ])
            )

        self.fc1 = nn.Linear(self.n_channels * 6 * 6, self.hidden_units)                                      
        self.fc2 = nn.Linear(self.hidden_units, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.lbc_blocks(out)
        out = torch.nn.functional.max_pool2d(out, kernel_size=6,padding=2)                      
        
        out = out.view(-1, self.n_channels * 6 * 6)                                             

        out = torch.relu(self.fc1(out))
        out = self.fc2(out)

        return out

## Training and validating functions

In [8]:
from datetime import datetime as datetime

def training_loop(n_epochs, model, loss_fn, train_loader, optimizer, scheduler=None):
    for epoch in range(1, n_epochs+1):
        train_loss = 0
        iter = 0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            out = model(imgs)
            loss = loss_fn(out, labels)

            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            
            iter += 1

        if scheduler is not None:
            scheduler.step()

        print("%s Epoch: %d, Loss: %f" % (datetime.now(), epoch, train_loss/len(train_loader)))

In [9]:
def validate(model, train_loader, val_loader):
    for season, loader in [('Train', train_loader), ('Validation', val_loader)]:
        total = 0
        correct = 0
        for imgs, labels in loader:
            imgs = imgs.to(device=device)
            labels = labels.to(device=device)
            with torch.no_grad():
                out = model(imgs)
                _, pred = torch.max(out, dim=1)

                total += imgs.shape[0]
                correct += int((pred == labels).sum())

        print("%s accuracy: %f" % (season, correct/total*100))

## Training

In [10]:
model = NetLBC(lbc_filters=512, n_channels=16, n_blocks=10, sparsity=0.1, hidden_units=384).to(device=device)

In [11]:
validate(model, train_loader, val_loader)

Train accuracy: 10.854000
Validation accuracy: 10.380000


In [12]:
loss = nn.CrossEntropyLoss()
learnable_parameters = []
for _, param in model.named_parameters():
    learnable_parameters.append(param)
optimizer = torch.optim.SGD(params=learnable_parameters, lr=1e-4, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [13]:
training_loop(n_epochs=270,
              model=model,
              loss_fn=loss,
              train_loader=train_loader,
              optimizer=optimizer,
              scheduler=scheduler)

Have already trained and saved the model, so just I'm just showing the results here. 

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
model.load_state_dict(torch.load('/content/drive/MyDrive/LBCNN_1.1.pt'))

<All keys matched successfully>

In [16]:
validate(model, train_loader, val_loader)

Train accuracy: 91.330000
Validation accuracy: 79.230000
